In [6]:
colab = False

if colab == True:
    !pip install nnunetv2
    !pip install torchio
    !git clone https://github.com/MIC-DKFZ/nnUNet.git
    !git clone https://github.com/NVIDIA/apex

In [7]:
import os
import torch
from torch.utils.data import random_split
import torchio as tio
from pathlib import Path
import importlib
from utils import validate_segmentation, plot_prediction, dice_score
import shutil
from collections import OrderedDict
import json
import numpy as np
import nnunetv2

import torchio_utils
importlib.reload(torchio_utils)
from torchio_utils import torchio_compose_train, plot_example

if colab:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)
    root_dir = '/content/drive/My Drive/tumor-segmentation'
    torch.cuda.empty_cache()

    training_size =1000

else:
    training_size = 200

path_to_mask = 'data/patients/labels/'
path_to_imgs = 'data/patients/imgs/'
path_to_controls = 'data/controls/imgs/' #healthy individuals (no tumors)
dataset_path = 'data.zip'
dataset_dir_name = 'data'
dataset_dir = Path(dataset_dir_name)

if not dataset_dir.is_dir():
    !curl --silent --output {dataset_path} --location {dataset_url}
    !unzip -qq {dataset_path}
!tree -d {dataset_dir_name}

data
├── controls
│   └── imgs
└── patients
    ├── imgs
    └── labels

5 directories


In [8]:
images_dir = dataset_dir / 'patients/imgs'
labels_dir = dataset_dir / 'patients/labels'
controls_dir = dataset_dir / 'controls/imgs'
image_paths = sorted(images_dir.glob('*.png'))
label_paths = sorted(labels_dir.glob('*.png'))
control_paths = sorted(controls_dir.glob('*.png'))

dataset = torchio_compose_train(image_paths, label_paths, control_paths, 
                                cropsize=(400,991), train_size = training_size,
                                save_training_dataset = True)

Found 182 subjects


  0%|          | 0/181 [00:00<?, ?it/s]

Landmarks: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.33322500e-02 6.22397969e-01 9.23236779e+00
 1.72545437e+01 2.07381415e+01 2.53002322e+01 4.03121636e+01
 1.00000000e+02]


/home/asp/anaconda3/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Input image is 2D, but "2" is in axes: (0, 1, 2)
  transformed = self.apply_transform(subject)


In [11]:
!nnUNetv2_plan_and_preprocess -d nnUNet_raw --verify_dataset_integrity

/home/asp/anaconda3/bin/python: symbol lookup error: /home/asp/anaconda3/lib/python3.11/site-packages/mkl/../../../libmkl_intel_thread.so.2: undefined symbol: omp_get_num_procs
